In [1]:
import pandas as pd
import numpy as np
import os
import cv2
import timm
import time
import torch
import torchvision
import skimage
import wandb
import hiplot
import neptune.new as neptune
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from tqdm import tqdm
from PIL import Image
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
class MaskDataset(Dataset):
    def __init__(self,labeldir,imgdir,transform):
        self.labeldata=pd.read_csv(labeldir)
        self.rootdir= imgdir
        self.transform=transform
    def __getitem__(self,index):
        img_path=os.path.join(self.rootdir,self.labeldata.iloc[index,0])
        image=skimage.io.imread(img_path)
        image=self.transform(image)
        label=torch.tensor(self.labeldata.iloc[index,1])
        return image,label
    def __len__(self):
        return self.labeldata.shape[0]

In [7]:
trainlabel="D:\\python\\CIS473\\CIS-473-Machine-Learning\\mask_data\\labels\\train_label.csv"
traindir="D:\\python\\CIS473\\CIS-473-Machine-Learning\\mask_data\\traindata\\"
testlabel="D:\\python\\CIS473\\CIS-473-Machine-Learning\\mask_data\\labels\\test_label.csv"
testdir="D:\\python\\CIS473\\CIS-473-Machine-Learning\\mask_data\\testdata\\"

transform=transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
])
batch_size=1
traindata=MaskDataset(trainlabel,traindir,transform)
train=DataLoader(dataset=traindata,batch_size=batch_size,shuffle=True)
testdata=MaskDataset(testlabel,testdir,transform)
test=DataLoader(dataset=testdata,batch_size=batch_size,shuffle=True)
#iter(train).next()[0].shape

1999

In [5]:
device=torch.device("cuda")
nepochs=10
learning_rate=0.01

res34 = timm.create_model('resnet34')
res34.fc=nn.Linear(512,2)
res34.add_module("Sig",nn.Softmax())
res34 = res34.to(device)
LossFunction = nn.CrossEntropyLoss()

In [8]:
loss_history = []
optimizer = optim.SGD(res34.parameters(), lr=learning_rate)
for epoch in tqdm(range(nepochs),desc="Epoch",colour="#26B69C"):
    for data,label in tqdm(train,colour="#26B69C",mininterval=30):
        
        optimizer.zero_grad()
        data = data.to(device)
        label=label.to(device)
        out = res34(data)
        #print(out,label)
        loss = LossFunction(out,label)
        loss.backward()
        optimizer.step()
        loss_history.append(loss.item())
    print(f"Epoch {epoch}: loss: {loss.item()}")

Epoch:  10%|███████▌                                                                    | 1/10 [01:31<13:45, 91.74s/it]

Epoch 0: loss: 0.9074983596801758



Epoch:  20%|███████████████▏                                                            | 2/10 [03:01<12:02, 90.32s/it]

Epoch 1: loss: 0.014398251660168171



Epoch:  30%|██████████████████████▊                                                     | 3/10 [04:30<10:27, 89.69s/it]

...

Epoch 7: loss: 0.00014804698002990335



Epoch:  90%|████████████████████████████████████████████████████████████████████▍       | 9/10 [13:03<01:25, 85.77s/it]

Epoch 8: loss: 0.003333846339955926



Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 10/10 [14:29<00:00, 86.96s/it]

Epoch 9: loss: 0.00020323118951637298


In [27]:
correct=0
total=0
with torch.no_grad():
    for data,label in tqdm(test,colour="#26B69C",mininterval=20):
        data = data.to(device)
        label=label.to(device)
        if data.shape==torch.Size([1, 3, 256, 256]):
            out=res34(data)
            pred=torch.argmax(out)
            if pred==label:
                correct+=1
            total+=1
print("Accuracy:",round(correct/total,3))
print(correct,total)

100%|██████████████████████████████████████████████████████████████████████████████| 1999/1999 [00:25<00:00, 79.01it/s]

Accuracy: 0.935
1863 1992


In [14]:
FILE="D:\\python\\CIS473\\CIS-473-Machine-Learning\\mask_data\\models\\model.pth"
torch.save(res34,FILE)